In [30]:
import os
import shutil
import cv2
import math
import random
import numpy as np
import datetime as dt
import tensorflow
import keras
from collections import deque
import matplotlib.pyplot as plt
plt.style.use("seaborn")

%matplotlib inline

from keras.layers import *
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

C:\Users\teck1\AppData\Local\Temp\ipykernel_13808\2821860772.py:12: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn")


In [31]:
import tensorflow as tf
imageheight, imagewidth = 64, 64
seq = 30
DATASET = "C:/Projects/GitHub/ai/artifacts/LCRM_data/final"

CLASSES_LIST = ["fighting", "running", "walking"]

seed_constant = 5
np.random.seed(seed_constant)
random.seed(seed_constant)
tensorflow.random.set_seed(seed_constant)

In [32]:
def frames_extraction(vid_path):
  frames_list = []

  #read vid file
  video_reader = cv2.VideoCapture(vid_path)

  #get total frames in vid
  video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

  #calculate interval for frames to be added
  skip_frames_window = max(int(video_frames_count/seq), 1)

  for frame_counter in range(seq):
    video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

    success, frame = video_reader.read()

    if not success:
      break

    #resize
    resized_frame = cv2.resize(frame, (imageheight, imagewidth))
    normalize_frame = resized_frame / 255

    frames_list.append(normalize_frame)

  video_reader.release()

  return frames_list

In [33]:
def create_dataset():
    features = []
    labels = []
    video_files_paths = []
    
    for class_index, class_name in enumerate(CLASSES_LIST):
        files_list = os.listdir(os.path.join(DATASET, class_name))
        
        for file_name in files_list:
            #get directory of the class
            video_file_path = os.path.join(DATASET, class_name, file_name)
            # Extract the frames of the video file.
            frames = frames_extraction(video_file_path)
            if len(frames) == seq:
                # Append the data to their repective lists.
                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)
                
    features = np.asarray(features)
    labels = np.array(labels)  

    return features, labels, video_files_paths

In [34]:
# Create the dataset.
features, labels, video_files_paths = create_dataset()

In [35]:
# convert labels into one-hot-encoded vectors
one_hot_encoded_labels = to_categorical(labels)

In [36]:
from sklearn.model_selection import train_test_split
# Split the Data into Train ( 90% ) and Test Set ( 10% ).
features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels, test_size = 0.1, shuffle = True, random_state = 42)
print(features_train.shape,labels_train.shape)
print(features_test.shape, labels_test.shape)

(270, 30, 64, 64, 3) (270, 3)
(30, 30, 64, 64, 3) (30, 3)


In [37]:
def create_LRCN_model():
    '''
    This function will construct the required LRCN model.
    Returns:
        model: It is the required constructed LRCN model.
    '''

    # We will use a Sequential model for model construction.
    model = Sequential()
    
    # Define the Model Architecture.
    ########################################################################################################################
    
    model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same',activation = 'relu'), input_shape = (seq, imageheight, imagewidth, 3)))
    model.add(TimeDistributed(MaxPooling2D((4, 4))))
    
    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((4, 4))))
    
    model.add(TimeDistributed(Conv2D(128, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    
    model.add(TimeDistributed(Conv2D(256, (2, 2), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
                                      
    model.add(TimeDistributed(Flatten()))
                                      
    model.add(LSTM(32))
                                      
    model.add(Dense(len(CLASSES_LIST), activation = 'softmax'))

    ########################################################################################################################

    # Display the models summary.
    model.summary()
    
    # Return the constructed LRCN model.
    return model

In [38]:
#Create model Instance
model = create_LRCN_model()

# Create an Instance of Early Stopping Callback.
early_stopping_callback = EarlyStopping(monitor = 'accuracy', patience = 10, mode = 'max', restore_best_weights = True)
 
# Compile the model and specify loss function, optimizer and metrics to the model.
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])
 
# Start training the model.
model_training_history = model.fit(x = features_train, y = labels_train, epochs = 70, batch_size = 16 , shuffle = True, validation_split = 0.25, callbacks = [early_stopping_callback])


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_18 (TimeD  (None, 30, 64, 64, 32)    896       
 istributed)                                                     
                                                                 
 time_distributed_19 (TimeD  (None, 30, 16, 16, 32)    0         
 istributed)                                                     
                                                                 
 time_distributed_20 (TimeD  (None, 30, 16, 16, 64)    18496     
 istributed)                                                     
                                                                 
 time_distributed_21 (TimeD  (None, 30, 4, 4, 64)      0         
 istributed)                                                     
                                                                 
 time_distributed_22 (TimeD  (None, 30, 4, 4, 128)    

In [39]:
model_evaluation_history = model.evaluate(features_test, labels_test)

1/1 [==============================] - 0s 265ms/step - loss: 0.7632 - accuracy: 0.7667


In [40]:
# Calculate Accuracy On Test Dataset
acc = 0
for i in range(len(features_test)):
  predicted_label = np.argmax(model.predict(np.expand_dims(features_test[i],axis =0))[0])
  actual_label = np.argmax(labels_test[i])
  if predicted_label == actual_label:
      acc += 1
acc = (acc * 100)/len(labels_test)
print("Accuracy =",acc)

1/1 [==============================] - 0s 88ms/step
Accuracy = 76.66666666666667


In [ ]:
model.save('artifacts/training/lcrm_model.h5', include_optimizer=True)